Code to accompany Machine Learning Recipes #8. We'll write a Decision Tree Classifier, in pure Python. Below each of the methods, I've written a little demo to help explain what it does.

In [1]:
# For Python 2 / 3 compatability
from __future__ import print_function
import pandas as pd

In [2]:
# Toy dataset.
# Format: each row is an example.
# The last column is the label.
# The first two columns are features.
# Feel free to play with it by adding more features & examples.
# Interesting note: I've written this so the 2nd and 5th examples
# have the same features, but different labels - so we can see how the
# tree handles this case.
training_data = [
    [865423,M,24.25,20.2,166.2,1761,0.1447,0.2867,0.4268,0.2012,0.2655,0.06877,1.509,3.12,9.807,233,0.02333,0.09806,0.1278,0.01822,0.04547,0.009875,26.02,23.99,180.9,2073,0.1696,0.4244,0.5803,0.2248,0.3222,0.08009],
    [865432,B,14.5,10.89,94.28,640.7,0.1101,0.1099,0.08842,0.05778,0.1856,0.06402,0.2929,0.857,1.928,24.19,0.003818,0.01276,0.02882,0.012,0.0191,0.002808,15.7,15.98,102.8,745.5,0.1313,0.1788,0.256,0.1221,0.2889,0.08006],
    [865468,B,13.37,16.39,86.1,553.5,0.07115,0.07325,0.08092,0.028,0.1422,0.05823,0.1639,1.14,1.223,14.66,0.005919,0.0327,0.04957,0.01038,0.01208,0.004076,14.26,22.75,91.99,632.1,0.1025,0.2531,0.3308,0.08978,0.2048,0.07628],
    [86561,B,13.85,17.21,88.44,588.7,0.08785,0.06136,0.0142,0.01141,0.1614,0.0589,0.2185,0.8561,1.495,17.91,0.004599,0.009169,0.009127,0.004814,0.01247,0.001708,15.49,23.58,100.3,725.9,0.1157,0.135,0.08115,0.05104,0.2364,0.07182],
    [866083,M,13.61,24.69,87.76,572.6,0.09258,0.07862,0.05285,0.03085,0.1761,0.0613,0.231,1.005,1.752,19.83,0.004088,0.01174,0.01796,0.00688,0.01323,0.001465,16.89,35.64,113.2,848.7,0.1471,0.2884,0.3796,0.1329,0.347,0.079],
    [866203,M,19,18.91,123.4,1138,0.08217,0.08028,0.09271,0.05627,0.1946,0.05044,0.6896,1.342,5.216,81.23,0.004428,0.02731,0.0404,0.01361,0.0203,0.002686,22.32,25.73,148.2,1538,0.1021,0.2264,0.3207,0.1218,0.2841,0.06541],
    [866458,B,15.1,16.39,99.58,674.5,0.115,0.1807,0.1138,0.08534,0.2001,0.06467,0.4309,1.068,2.796,39.84,0.009006,0.04185,0.03204,0.02258,0.02353,0.004984,16.11,18.33,105.9,762.6,0.1386,0.2883,0.196,0.1423,0.259,0.07779],
    [866674,M,19.79,25.12,130.4,1192,0.1015,0.1589,0.2545,0.1149,0.2202,0.06113,0.4953,1.199,2.765,63.33,0.005033,0.03179,0.04755,0.01043,0.01578,0.003224,22.63,33.58,148.7,1589,0.1275,0.3861,0.5673,0.1732,0.3305,0.08465],
    [866714,B,12.19,13.29,79.08,455.8,0.1066,0.09509,0.02855,0.02882,0.188,0.06471,0.2005,0.8163,1.973,15.24,0.006773,0.02456,0.01018,0.008094,0.02662,0.004143,13.34,17.81,91.38,545.2,0.1427,0.2585,0.09915,0.08187,0.3469,0.09241],
    [8670,M,15.46,19.48,101.7,748.9,0.1092,0.1223,0.1466,0.08087,0.1931,0.05796,0.4743,0.7859,3.094,48.31,0.00624,0.01484,0.02813,0.01093,0.01397,0.002461,19.26,26,124.9,1156,0.1546,0.2394,0.3791,0.1514,0.2837,0.08019],
    [86730502,M,16.16,21.54,106.2,809.8,0.1008,0.1284,0.1043,0.05613,0.216,0.05891,0.4332,1.265,2.844,43.68,0.004877,0.01952,0.02219,0.009231,0.01535,0.002373,19.47,31.68,129.7,1175,0.1395,0.3055,0.2992,0.1312,0.348,0.07619],
    [867387,B,15.71,13.93,102,761.7,0.09462,0.09462,0.07135,0.05933,0.1816,0.05723,0.3117,0.8155,1.972,27.94,0.005217,0.01515,0.01678,0.01268,0.01669,0.00233,17.5,19.25,114.3,922.8,0.1223,0.1949,0.1709,0.1374,0.2723,0.07071],
    [867739,M,18.45,21.91,120.2,1075,0.0943,0.09709,0.1153,0.06847,0.1692,0.05727,0.5959,1.202,3.766,68.35,0.006001,0.01422,0.02855,0.009148,0.01492,0.002205,22.52,31.39,145.6,1590,0.1465,0.2275,0.3965,0.1379,0.3109,0.0761],
    [868202,M,12.77,22.47,81.72,506.3,0.09055,0.05761,0.04711,0.02704,0.1585,0.06065,0.2367,1.38,1.457,19.87,0.007499,0.01202,0.02332,0.00892,0.01647,0.002629,14.49,33.37,92.04,653.6,0.1419,0.1523,0.2177,0.09331,0.2829,0.08067],
    [868223,B,11.71,16.67,74.72,423.6,0.1051,0.06095,0.03592,0.026,0.1339,0.05945,0.4489,2.508,3.258,34.37,0.006578,0.0138,0.02662,0.01307,0.01359,0.003707,13.33,25.48,86.16,546.7,0.1271,0.1028,0.1046,0.06968,0.1712,0.07343],
    [868682,B,11.43,15.39,73.06,399.8,0.09639,0.06889,0.03503,0.02875,0.1734,0.05865,0.1759,0.9938,1.143,12.67,0.005133,0.01521,0.01434,0.008602,0.01501,0.001588,12.32,22.02,79.93,462,0.119,0.1648,0.1399,0.08476,0.2676,0.06765],
    [868826,M,14.95,17.57,96.85,678.1,0.1167,0.1305,0.1539,0.08624,0.1957,0.06216,1.296,1.452,8.419,101.9,0.01,0.0348,0.06577,0.02801,0.05168,0.002887,18.55,21.43,121.4,971.4,0.1411,0.2164,0.3355,0.1667,0.3414,0.07147],
    [868871,B,11.28,13.39,73,384.8,0.1164,0.1136,0.04635,0.04796,0.1771,0.06072,0.3384,1.343,1.851,26.33,0.01127,0.03498,0.02187,0.01965,0.0158,0.003442,11.92,15.77,76.53,434,0.1367,0.1822,0.08669,0.08611,0.2102,0.06784],
    [868999,B,9.738,11.97,61.24,288.5,0.0925,0.04102,0,0,0.1903,0.06422,0.1988,0.496,1.218,12.26,0.00604,0.005656,0,0,0.02277,0.00322,10.62,14.1,66.53,342.9,0.1234,0.07204,0,0,0.3105,0.08151],
    [869104,M,16.11,18.05,105.1,813,0.09721,0.1137,0.09447,0.05943,0.1861,0.06248,0.7049,1.332,4.533,74.08,0.00677,0.01938,0.03067,0.01167,0.01875,0.003434,19.92,25.27,129,1233,0.1314,0.2236,0.2802,0.1216,0.2792,0.08158],
    [869218,B,11.43,17.31,73.66,398,0.1092,0.09486,0.02031,0.01861,0.1645,0.06562,0.2843,1.908,1.937,21.38,0.006664,0.01735,0.01158,0.00952,0.02282,0.003526,12.78,26.76,82.66,503,0.1413,0.1792,0.07708,0.06402,0.2584,0.08096],
    [869224,B,12.9,15.92,83.74,512.2,0.08677,0.09509,0.04894,0.03088,0.1778,0.06235,0.2143,0.7712,1.689,16.64,0.005324,0.01563,0.0151,0.007584,0.02104,0.001887,14.48,21.82,97.17,643.8,0.1312,0.2548,0.209,0.1012,0.3549,0.08118],
    [869254,B,10.75,14.97,68.26,355.3,0.07793,0.05139,0.02251,0.007875,0.1399,0.05688,0.2525,1.239,1.806,17.74,0.006547,0.01781,0.02018,0.005612,0.01671,0.00236,11.95,20.72,77.79,441.2,0.1076,0.1223,0.09755,0.03413,0.23,0.06769]
]

In [3]:
# Column labels.
header = ["id","clamp_thickness","cell_size","cell_shape","marginal_adhesion","single_epith_cell","bare_nuclei","bland_chromatin","normal_nucleoli","mitoses","class"]
df = pd.DataFrame(training_data, columns=header)
print(df)

         id  clamp_thickness  cell_size  cell_shape  marginal_adhesion  \
0   1000025                5          1           1                  1   
1   1002945                5          4           4                  5   
2   1015425                3          1           1                  1   
3   1016277                6          8           8                  1   
4   1017023                4          1           1                  3   
5   1017122                8         10          10                  8   
6   1018099                1          1           1                  1   
7   1018561                2          1           2                  1   
8   1033078                2          1           1                  1   
9   1033078                4          2           1                  1   
10  1035283                1          1           1                  1   
11  1036172                2          1           1                  1   
12  1041801                5          

In [4]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [5]:
#######
# Demo:
unique_vals(training_data, 0)
# unique_vals(training_data, 1)
#######

{1000025,
 1002945,
 1015425,
 1016277,
 1017023,
 1017122,
 1018099,
 1018561,
 1033078,
 1035283,
 1036172,
 1041801,
 1043999,
 1044572,
 1047630,
 1048672,
 1049815,
 1050670,
 1050718}

In [6]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [7]:
#######
# Demo:
class_counts(training_data)
#######

{2: 15, 4: 5}

In [8]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [9]:
#######
# Demo:
is_numeric(7)
# is_numeric("Red")
#######

True

In [10]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [11]:
#######
# Demo:
# Let's write a question for a numeric attribute
q = Question(1, 5)
q

Is clamp_thickness >= 5?

In [12]:
# Let's pick an example from the training set...
example = training_data[0]
# ... and see if it matches the question
q.match(example) # this will be true, since the first example is Green.
#######

True

In [13]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [14]:
#######
# Demo:
# Let's partition the training data based on whether rows are Red.
true_rows, false_rows = partition(training_data, Question(5, 4))
# This will contain all the 'Red' rows.
true_rows

[[1002945, 5, 4, 4, 5, 7, 10, 3, 2, 1, 2],
 [1017122, 8, 10, 10, 8, 7, 10, 9, 7, 1, 4],
 [1044572, 8, 7, 5, 10, 7, 9, 5, 5, 4, 4],
 [1047630, 7, 4, 6, 4, 6, 1, 4, 3, 1, 4],
 [1050670, 10, 7, 7, 6, 4, 10, 4, 1, 2, 4]]

In [15]:
# This will contain everything else.
false_rows
#######

[[1000025, 5, 1, 1, 1, 2, 1, 3, 1, 1, 2],
 [1015425, 3, 1, 1, 1, 2, 2, 3, 1, 1, 2],
 [1016277, 6, 8, 8, 1, 3, 4, 3, 7, 1, 2],
 [1017023, 4, 1, 1, 3, 2, 1, 3, 1, 1, 2],
 [1018099, 1, 1, 1, 1, 2, 10, 3, 1, 1, 2],
 [1018561, 2, 1, 2, 1, 2, 1, 3, 1, 1, 2],
 [1033078, 2, 1, 1, 1, 2, 1, 1, 1, 5, 2],
 [1033078, 4, 2, 1, 1, 2, 1, 2, 1, 1, 2],
 [1035283, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2],
 [1036172, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2],
 [1041801, 5, 3, 3, 3, 2, 3, 4, 4, 1, 4],
 [1043999, 1, 1, 1, 1, 2, 3, 3, 1, 1, 2],
 [1048672, 4, 1, 1, 1, 2, 1, 2, 1, 1, 2],
 [1049815, 4, 1, 1, 1, 2, 1, 3, 1, 1, 2],
 [1050718, 6, 1, 1, 1, 2, 1, 3, 1, 1, 2]]

In [16]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [17]:
#######
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = [['F'],
              ['F']]
# this will return 0
gini(no_mixing)

0.0

In [18]:
# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = [['M'],
               ['F']]
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
gini(some_mixing)

0.5

In [19]:
# Now, we'll look at a dataset with many different labels
lots_of_mixing = [['M'],
                  ['F'],
                  ['T']]
# This will return 0.8
gini(lots_of_mixing)
#######

0.6666666666666665

In [20]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [21]:
#######
# Demo:
# Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
current_uncertainty

0.375

In [22]:
# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(5, 4))
info_gain(true_rows, false_rows, current_uncertainty)

0.20166666666666674

In [23]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(training_data, Question(2,6))
info_gain(true_rows, false_rows, current_uncertainty)

0.12499999999999997

In [24]:
# It looks like we learned more using 'Red' (0.37), than 'Green' (0.14).
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.
true_rows, false_rows = partition(training_data, Question(5,4))

# Here, the true_rows contain only 'Grapes'.
true_rows

[[1002945, 5, 4, 4, 5, 7, 10, 3, 2, 1, 2],
 [1017122, 8, 10, 10, 8, 7, 10, 9, 7, 1, 4],
 [1044572, 8, 7, 5, 10, 7, 9, 5, 5, 4, 4],
 [1047630, 7, 4, 6, 4, 6, 1, 4, 3, 1, 4],
 [1050670, 10, 7, 7, 6, 4, 10, 4, 1, 2, 4]]

In [25]:
# And the false rows contain two types of fruit. Not too bad.
false_rows

[[1000025, 5, 1, 1, 1, 2, 1, 3, 1, 1, 2],
 [1015425, 3, 1, 1, 1, 2, 2, 3, 1, 1, 2],
 [1016277, 6, 8, 8, 1, 3, 4, 3, 7, 1, 2],
 [1017023, 4, 1, 1, 3, 2, 1, 3, 1, 1, 2],
 [1018099, 1, 1, 1, 1, 2, 10, 3, 1, 1, 2],
 [1018561, 2, 1, 2, 1, 2, 1, 3, 1, 1, 2],
 [1033078, 2, 1, 1, 1, 2, 1, 1, 1, 5, 2],
 [1033078, 4, 2, 1, 1, 2, 1, 2, 1, 1, 2],
 [1035283, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2],
 [1036172, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2],
 [1041801, 5, 3, 3, 3, 2, 3, 4, 4, 1, 4],
 [1043999, 1, 1, 1, 1, 2, 3, 3, 1, 1, 2],
 [1048672, 4, 1, 1, 1, 2, 1, 2, 1, 1, 2],
 [1049815, 4, 1, 1, 1, 2, 1, 3, 1, 1, 2],
 [1050718, 6, 1, 1, 1, 2, 1, 3, 1, 1, 2]]

In [26]:
# On the other hand, partitioning by Green doesn't help so much.
true_rows, false_rows = partition(training_data, Question(2,6))

# We've isolated one apple in the true rows.
true_rows

[[1016277, 6, 8, 8, 1, 3, 4, 3, 7, 1, 2],
 [1017122, 8, 10, 10, 8, 7, 10, 9, 7, 1, 4],
 [1044572, 8, 7, 5, 10, 7, 9, 5, 5, 4, 4],
 [1050670, 10, 7, 7, 6, 4, 10, 4, 1, 2, 4]]

In [27]:
# But, the false-rows are badly mixed up.
false_rows
#######

[[1000025, 5, 1, 1, 1, 2, 1, 3, 1, 1, 2],
 [1002945, 5, 4, 4, 5, 7, 10, 3, 2, 1, 2],
 [1015425, 3, 1, 1, 1, 2, 2, 3, 1, 1, 2],
 [1017023, 4, 1, 1, 3, 2, 1, 3, 1, 1, 2],
 [1018099, 1, 1, 1, 1, 2, 10, 3, 1, 1, 2],
 [1018561, 2, 1, 2, 1, 2, 1, 3, 1, 1, 2],
 [1033078, 2, 1, 1, 1, 2, 1, 1, 1, 5, 2],
 [1033078, 4, 2, 1, 1, 2, 1, 2, 1, 1, 2],
 [1035283, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2],
 [1036172, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2],
 [1041801, 5, 3, 3, 3, 2, 3, 4, 4, 1, 4],
 [1043999, 1, 1, 1, 1, 2, 3, 3, 1, 1, 2],
 [1047630, 7, 4, 6, 4, 6, 1, 4, 3, 1, 4],
 [1048672, 4, 1, 1, 1, 2, 1, 2, 1, 1, 2],
 [1049815, 4, 1, 1, 1, 2, 1, 3, 1, 1, 2],
 [1050718, 6, 1, 1, 1, 2, 1, 3, 1, 1, 2]]

In [28]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [29]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is bland_chromatin >= 4?

In [30]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [31]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [32]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [33]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [34]:
my_tree = build_tree(training_data)

In [35]:
print_tree(my_tree)

Is bland_chromatin >= 4?
--> True:
  Predict {4: 5}
--> False:
  Predict {2: 15}


In [36]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [37]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(training_data[0], my_tree)
#######

{2: 15}

In [38]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [39]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(training_data[0], my_tree))
#######

{2: '100%'}

In [40]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(training_data[1], my_tree))
#######

{2: '100%'}

In [43]:
# Evaluate
testing_data = [
[1239232,3,3,2,6,3,3,3,5,1,2],
[1239347,8,7,8,5,10,10,7,2,1,4],
[1239967,1,1,1,1,2,1,2,1,1,2],
[1240337,5,2,2,2,2,2,3,2,2,2],
[1253505,2,3,1,1,5,1,1,1,1,2],
[1255384,3,2,2,3,2,3,3,1,1,2],
[1257200,10,10,10,7,10,10,8,2,1,4],
[1257648,4,3,3,1,2,1,3,3,1,2],
[1257815,5,1,3,1,2,1,2,1,1,2],
[1257938,3,1,1,1,2,1,1,1,1,2],
[1258549,9,10,10,10,10,10,10,10,1,4]
]

In [44]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: 2. Predicted: {2: '100%'}
Actual: 4. Predicted: {4: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 4. Predicted: {4: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 2. Predicted: {2: '100%'}
Actual: 4. Predicted: {4: '100%'}
